In [ ]:
import time
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/topicos-dataset"

# Pre-procesado de Meta.csv

In [ ]:
ubigeo = pd.read_csv(f"TB_UBIGEOS.csv", usecols=[0, 1, 2, 4, 6, 7])

ubigeo.head()

,id_ubigeo,ubigeo_reniec,ubigeo_inei,departamento,provincia,distrito
0,1,10101.0,10101.0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS
1,2,10102.0,10102.0,AMAZONAS,CHACHAPOYAS,ASUNCION
2,3,10103.0,10103.0,AMAZONAS,CHACHAPOYAS,BALSAS
3,4,10104.0,10104.0,AMAZONAS,CHACHAPOYAS,CHETO
4,5,10105.0,10105.0,AMAZONAS,CHACHAPOYAS,CHILIQUIN


TB_UBIGEOS.csv tiene presencia de NA tanto en RENIEC como en INEI

In [ ]:
ubigeo[ ubigeo.ubigeo_reniec.isna() ]

,id_ubigeo,ubigeo_reniec,ubigeo_inei,departamento,provincia,distrito
1500,1886,NaN,150144.0,LIMA,LIMA,SANTA MARIA DE HUACHIPA


In [ ]:
ubigeo[ ubigeo.ubigeo_inei.isna() ]

,id_ubigeo,ubigeo_reniec,ubigeo_inei,departamento,provincia,distrito
1497,1883,170107.0,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO


Luego de pre-procesar Meta.csv verificaremos si es viable usar join usando INEI o RENIEC

In [ ]:
meta = pd.read_csv(f"{path}/Meta.csv")

meta.head()

,Ubigeo_INEI,Ubigeo_RENIEC,Edad,Sexo,Meta_2021,Meta_2022
0,40413,40512,28,F,56,56
1,160211,150211,59,F,14,26
2,150204,140904,43,M,190,172
3,210701,200601,68,M,45,49
4,110401,100501,13,F,95,73


In [ ]:
edad_names = ["5-11", "12-17", "18-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80 >"]
edad_grupo = {e:(i+1) for i, e in enumerate(edad_names)}

In [ ]:
meta['Grupo Edad'] = pd.cut(
    meta['Edad'],
    bins=[5, 11, 17, 29, 39, 49, 59, 69, 79, 200],
    labels= edad_grupo.values(),
    right=False
)

meta.head()

,Ubigeo_INEI,Ubigeo_RENIEC,Edad,Sexo,Meta_2021,Meta_2022,Grupo Edad
0,40413,40512,28,F,56,56,3
1,160211,150211,59,F,14,26,7
2,150204,140904,43,M,190,172,5
3,210701,200601,68,M,45,49,7
4,110401,100501,13,F,95,73,2


Probando la existencia de registros (en Meta.csv) cuyo código no aparece en TB_UBIGEOS.csv

In [ ]:
meta[meta["Ubigeo_RENIEC"] == 170107]
# ESTE CODIGO RENIEC PRESENTA NA EN EL CSV DEL EQUIVALENTE INEI
# POR LO QUE NO SERIA POSIBLE HACER JOIN CON EL CODIGO INEI

,Ubigeo_INEI,Ubigeo_RENIEC,Edad,Sexo,Meta_2021,Meta_2022,Grupo Edad
34302,180199,170107,23,M,0,1,3
67062,180199,170107,41,F,1,0,5
77723,180199,170107,30,M,0,1,4
121656,180199,170107,29,M,1,0,4
152704,180199,170107,39,M,1,0,5
153990,180199,170107,20,M,0,1,3
176482,180199,170107,19,M,1,0,3
220495,180199,170107,42,F,0,1,5
332820,180199,170107,40,M,0,1,5


In [ ]:
meta[meta["Ubigeo_INEI"] == 150144]
# ESTE CODIGO INEI PRESENTA NA EN EL CSV DEL EQUIVALENTE RENIEC
# SIN EMBARGO, NO HAY NINGUN REGISTRO
# POR LO QUE NO ES PROBLEMA SI SE HACE JOIN CON EL CODIGO RENIEC

,Ubigeo_INEI,Ubigeo_RENIEC,Edad,Sexo,Meta_2021,Meta_2022,Grupo Edad


JOIN entre TB_UBIGEOS.csv y Meta.csv usando el código de ubigeo de RENIEC

In [ ]:
# ES NECESARIO CONVERTIR DE "Ubigeo_RENIEC" A "ubigeo_reniec" DE Meta.csv
meta.rename(columns = {'Ubigeo_RENIEC':'ubigeo_reniec'}, inplace = True)

In [ ]:
left_merged = pd.merge(meta, ubigeo, how="left", on=["ubigeo_reniec"])

left_merged.head()

,Ubigeo_INEI,ubigeo_reniec,Edad,Sexo,Meta_2021,Meta_2022,Grupo Edad,id_ubigeo,ubigeo_inei,departamento,provincia,distrito
0,40413,40512,28,F,56,56,3,397,40413.0,AREQUIPA,CASTILLA,URACA
1,160211,150211,59,F,14,26,7,1469,160211.0,LORETO,ALTO AMAZONAS,TENIENTE CESAR LOPEZ ROJAS
2,150204,140904,43,M,190,172,5,1328,150204.0,LIMA,BARRANCA,SUPE
3,210701,200601,68,M,45,49,7,1691,210701.0,PUNO,LAMPA,LAMPA
4,110401,100501,13,F,95,73,2,1023,110401.0,ICA,PALPA,PALPA


Comprobando que la columna con NA no haya afectado el join

In [ ]:
left_merged[left_merged["ubigeo_reniec"] == 170107].head()

,Ubigeo_INEI,ubigeo_reniec,Edad,Sexo,Meta_2021,Meta_2022,Grupo Edad,id_ubigeo,ubigeo_inei,departamento,provincia,distrito
34302,180199,170107,23,M,0,1,3,1883,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO
67062,180199,170107,41,F,1,0,5,1883,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO
77723,180199,170107,30,M,0,1,4,1883,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO
121656,180199,170107,29,M,1,0,4,1883,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO
152704,180199,170107,39,M,1,0,5,1883,NaN,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO


Guardar el dataset pre-procesado

In [ ]:
left_merged.drop(left_merged.columns[[2, 3, 7, 8]], axis=1).to_csv(f"{path}/MetaPreprocessed.csv", index=False)

Cargamos el dataset pre-procesado

In [ ]:
df = pd.read_csv(f"{path}/MetaPreprocessed.csv")

df.head()

,Ubigeo_INEI,ubigeo_reniec,Meta_2021,Meta_2022,Grupo Edad,departamento,provincia,distrito
0,40413,40512,56,56,3,AREQUIPA,CASTILLA,URACA
1,160211,150211,14,26,7,LORETO,ALTO AMAZONAS,TENIENTE CESAR LOPEZ ROJAS
2,150204,140904,190,172,5,LIMA,BARRANCA,SUPE
3,210701,200601,45,49,7,PUNO,LAMPA,LAMPA
4,110401,100501,95,73,2,ICA,PALPA,PALPA


# Dividiendo VacunaCovid_Resumen.csv por Departamento

Pandas demora aproximadamente 1 minuto en leer el archivo por completo. El tiempo se reduce considerablemente si se filtra una cierta cantidad de columnas, pero aun no es óptimo. Es por ello que divideremos el dataset en varios archivos

In [ ]:
start = time.time()
vacunas_todo_df = pd.read_csv(f"{path}/VacunaCovid_Resumen.csv", usecols=[0, 1, 3, 4, 5, 8, 11, 12, 13])
end = time.time()

print(end - start)

26.61271357536316


In [ ]:
vacunas_todo_df.head()

,Ubigeo_Inei,Ubigeo_Reniec,Departamento,Provincia,Distrito,Grupo Edad,Fecha,Tipo dosis,Dosis
0,70101,240101,CALLAO,CALLAO,CALLAO,1,28/01/2022,1,1
1,70105,240103,CALLAO,CALLAO,LA PUNTA,1,28/02/2022,1,1
2,70105,240103,CALLAO,CALLAO,LA PUNTA,1,7/04/2022,1,1
3,70105,240103,CALLAO,CALLAO,LA PUNTA,1,22/02/2022,1,2
4,70101,240101,CALLAO,CALLAO,CALLAO,1,25/04/2022,1,3


In [ ]:
df = vacunas_todo_df
departamentos = list( df["Departamento"].unique() )

[df[df["Departamento"] == dep].to_csv(f"{path}/VacunaCovid_Resumen/{dep}.csv", index=False) for dep in departamentos]

print("Done")

Done


In [ ]:
_selected_dep = "LIMA"

start = time.time()
df_lima = pd.read_csv(f"{path}/VacunaCovid_Resumen/{_selected_dep}.csv", parse_dates=["Fecha"])
end = time.time()

print(end - start)

1.8878297805786133


In [ ]:
df_lima.head()

,Ubigeo_Inei,Ubigeo_Reniec,Departamento,Provincia,Distrito,Grupo Edad,Fecha,Tipo dosis,Dosis
0,150141,140131,LIMA,LIMA,SURQUILLO,1,2022-01-25,1,1
1,150141,140131,LIMA,LIMA,SURQUILLO,1,2022-01-28,1,1
2,150121,140117,LIMA,LIMA,MAGDALENA VIEJA (PUEBLO LIBRE),1,2022-03-14,1,1
3,150121,140117,LIMA,LIMA,MAGDALENA VIEJA (PUEBLO LIBRE),1,2022-03-20,1,1
4,150130,140140,LIMA,LIMA,SAN BORJA,1,2022-03-21,1,1


In [ ]:
df_lima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677498 entries, 0 to 1677497
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Ubigeo_Inei    1677498 non-null  int64         
 1   Ubigeo_Reniec  1677498 non-null  int64         
 2   Departamento   1677498 non-null  object        
 3   Provincia      1677498 non-null  object        
 4   Distrito       1677498 non-null  object        
 5   Grupo Edad     1677498 non-null  int64         
 6   Fecha          1675808 non-null  datetime64[ns]
 7   Tipo dosis     1677498 non-null  int64         
 8   Dosis          1677498 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 115.2+ MB
